In [58]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [59]:
df = pd.read_csv("data/us_estdata.csv")
df

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,JOBS_EDU_BACH_ORIG,HH_COST_ORIG_ADJ,PINCP_ORIG_ADJ,HINCP_ORIG_ADJ,SMOCP_ORIG_ADJ,GRNTP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ,OWN_JOB_ORIG,TYPE_ORIG,CBSA_NAME_ORIG
0,0,P,2018GQ0103142,2,1,902,1,34,1013097,27,...,139711.0,0.205419,32500.0,114100.0,1986.5,1500.0,0.044023,0.0,0,367.0
1,1,P,2018HU1096416,9,1,10200,4,53,1013097,341,...,14092.0,0.180426,32000.0,79000.0,1222.0,1134.0,0.037921,5903.0,1,495.0
2,2,P,2018HU0306186,7,2,1400,3,48,1013097,43,...,4215.0,0.170502,30000.0,81100.0,1120.0,1048.0,0.064784,2964.0,0,101.0
3,3,P,2018HU0641097,7,1,1200,3,48,1013097,24,...,4446.0,0.149895,24000.0,59700.0,669.0,820.0,0.036782,0.0,2,95.0
4,4,P,2018HU1246748,5,1,7300,3,12,1013097,75,...,34392.0,0.172667,30500.0,75000.0,994.0,980.0,0.048288,21139.0,1,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63267,63267,P,2018HU0767933,9,2,3765,4,6,1013097,85,...,1116653.0,0.243800,24500.0,83150.0,1933.0,1480.0,0.054669,751192.0,0,23.0
63268,63268,P,2018HU0401617,9,1,7704,4,6,1013097,113,...,47644.0,0.210121,24000.0,77200.0,1463.0,1238.0,0.060292,19802.0,1,22.0
63269,63269,P,2018HU0527555,5,3,5100,3,37,1013097,158,...,8320.0,0.175161,17000.0,43200.0,568.5,640.0,0.052511,0.0,3,-1.0
63270,63270,P,2018HU1061510,9,2,8502,4,6,1013097,189,...,399143.0,0.197049,41600.0,148000.0,2617.5,2346.0,0.040629,64382.0,1,1.0


In [60]:
df["ORIGIN"].value_counts()

603700     1997
2500390     937
1703400     870
400100      794
4804600     682
           ... 
2201600      11
1801700      11
2202100      10
1304400      10
4806900      10
Name: ORIGIN, Length: 975, dtype: int64

In [61]:
df["MED"] = np.where(df["NAICS"] == "MED", 1, 0)
df["MFG"] = np.where(df["NAICS"] == "MFG", 1, 0)
df["RET"] = np.where(df["NAICS"] == "RET", 1, 0)
df["EDU"] = np.where(df["NAICS"] == "EDU", 1, 0)
df["ADM"] = np.where(df["NAICS"] == "ADM", 1, 0)
df["FOD"] = np.where(df["NAICS"] == "FOD", 1, 0)
df["PRF"] = np.where(df["NAICS"] == "PRF", 1, 0)
df["TRN"] = np.where(df["NAICS"] == "TRN", 1, 0)
df["SRV"] = np.where(df["NAICS"] == "SRV", 1, 0)
df["FIN"] = np.where(df["NAICS"] == "FIN", 1, 0)
df["WHL"] = np.where(df["NAICS"] == "WHL", 1, 0)
df["AGR"] = np.where(df["NAICS"] == "AGR", 1, 0)
df["PUB"] = np.where(df["NAICS"] == "PUB", 1, 0)
df["INF"] = np.where(df["NAICS"] == "INF", 1, 0)
df["ENT"] = np.where(df["NAICS"] == "ENT", 1, 0)
df["REL"] = np.where(df["NAICS"] == "REL", 1, 0)
df["UTL"] = np.where(df["NAICS"] == "UTL", 1, 0)
df["EXT"] = np.where(df["NAICS"] == "EXT", 1, 0)
df["MNG"] = np.where(df["NAICS"] == "MNG", 1, 0)
df["CON"] = np.where(df["NAICS"] == "CON", 1, 0)
df["NO_IND"] = np.where(df["NAICS"].isna(), 1, 0)

In [62]:
df["NO_IND"].sum()

17447

In [63]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [64]:
df["CHOSEN"].value_counts()

102500     106
5500100     85
5310200     84
5500700     74
1700202     73
          ... 
4804619      7
2701404      7
5541001      6
2601703      5
3500806      4
Name: CHOSEN, Length: 2336, dtype: int64

In [65]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 101): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [66]:
df["CHOSEN"].value_counts()

0    55261
1     8011
Name: CHOSEN, dtype: int64

In [67]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0        0
1        0
2        0
3        0
4        0
        ..
63267    0
63268    0
63269    0
63270    0
63271    0
Name: IN_COLLEGE, Length: 63272, dtype: int32

In [68]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [69]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [70]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGHONLY"] = np.where((df["SCHL"] >= 16) & (df["SCHL"] <= 17), 1, 0)
df["EDU_SOMECOLLEGE"] = np.where((df["SCHL"] >= 18) & (df["SCHL"] <= 20), 1, 0)
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [71]:
df["BEYOND_HS"] = np.where(df["SCHL"] > 17, 1, 0)

In [72]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [73]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [74]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [75]:
# df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
# df["child_old"].value_counts()
df["child"].value_counts()

0    43851
1    19421
Name: child, dtype: int64

In [76]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    62389
1      883
Name: REC_NO_MAR, dtype: int64

In [77]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    62261
1.0     1011
Name: MARHM_new, dtype: int64

In [78]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    33059
0    30213
Name: married_old, dtype: int64

In [79]:
df["MILITARY"] = np.where(df["MIL"] == 1, 1, 0)

In [80]:
# df["HH_COST"] = np.where(df["TEN"] == 3, df["GRNTP"], df["SMOCP"])

In [81]:
# up to debate still
df["AGR_EXT"] = np.where((df["AGR"] == 1) | (df["EXT"] == 1), 1, 0)
df["HIGH_ED"] = np.where((df["MED"] == 1) | (df["EDU"] == 1) | (df["PRF"] == 1) | (df["MED"] == 1) | (df["FIN"] == 1) | (df["INF"] == 1) | (df["MED"] == 1), 1, 0)
df["LICENSE"] = np.where((df["SRV"] == 1) | (df["REL"] == 1), 1, 0)
df["OTHER_JOB"] = np.where((df["AGR_EXT"] == 0) & (df["HIGH_ED"] == 0) & (df["LICENSE"] == 0) & (df["NO_IND"] == 0), 1, 0)

In [82]:
df["MICRO_adj_ORIG"] = np.where((df["TYPE_ORIG"] == 3) | (df["TYPE_ORIG"] == 2), 1, 0)
df["METRO"] = np.where(df["TYPE_ORIG"] == 1, 1, 0)
df["T34"] = np.where(df["TYPE_ORIG"] == 0, 1, 0)

In [83]:
for i in range(1, 101):
    key = "ALT{0}_".format(i)
    df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
    df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
    df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\1465497578.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\1465497578.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\1465497578.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [84]:
df["UNEMPLOYED"] = np.where((df["ESR"] == 3) | (df["ESR"] == 6), 1, 0)
df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\2921326413.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)


In [85]:
df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
df["OTHER_FAMILY"] = np.where((df["HHT"] == 2) | (df["HHT"] == 3), 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\3935644085.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\3935644085.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_15132\3935644085.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [86]:
df.loc[df["CBSA_NAME_ORIG"] == -1, "CBSA_NAME_ORIG"] = -2

In [87]:
# df["INTERNAL"] = df["ORIGIN"] == df["CHOSEN"]

In [88]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [89]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [90]:
c_move = Beta("c_move", 0, None, None, 0)

In [ ]:
# Staying Choice Parameters to be Estimated
c_stay_age_18_22 = Beta("c_stay_age_18_22", 0.0621, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", -0.113, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", -0.0233, None, None, 0)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0.149, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 1.43, None, None, 0)
c_stay_edu_beyondhs = Beta('c_stay_edu_beyondhs', -0.0944, None, None, 0)
c_stay_child = Beta("c_stay_child", -0.278, None, None, 0)
c_stay = Beta("c_stay", 17.4, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0.185, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0.00000437, None, None, 0)
c_stay_college = Beta("c_stay_college", 0.706, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0.0288, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", -0.856, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", -0.359, None, None, 0)
c_stay_mil = Beta("c_stay_mil", -0.165, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 0.713, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0.573, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0.497, None, None, 0)
c_stay_ownjob = Beta("c_stay_ownjob", 1.78, None, None, 0)

In [ ]:
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_income = Beta("c_stay_income", 0, None, None, 0)

In [96]:
# defining the staying utility function
V0 = c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_beyondhs * BEYOND_HS + c_stay_child * child + c_stay + c_stay_dens * DENS_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY + c_stay_hh_val * HH_MED_VAL_ORIG + MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG + (c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ) / HINCP_ORIG_ADJ * 12 + c_stay_foreign * FOREIGN + c_stay_unemp_rate * UNEMP_PCT_ORIG_ADJ

In [98]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', -0.0000000241, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', -0.257, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp_rate", -2.14, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", -1.09, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0.0000679, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 3.25, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0.00000439, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 7.19, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 1.57, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0.00000305, None, None, 0)
c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 1.68, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 2.24, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 2.52, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0.000245, None, None, 0)

In [100]:
c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0.481, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0.582, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0.165, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0.881, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 1.06, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", -0.199, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0.724, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 1.52, None, None, 0)

In [101]:
c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 5.36, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 2.51, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 5.97, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 2.43, None, None, 0)

In [102]:
c_destchoice_samestate = Beta("c_destchoice_samestate", 6.33, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0.918, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0.511, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", -0.0000278, None, None, 0)

In [103]:
# birthstate > cbsa > state

In [104]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(100):
    num = i + 1
    initialization = "V{0} = log(ALT{0}_POP) + (CBSA_NAME_ORIG != ALT{0}_CBSA) * (c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1)) + (CBSA_NAME_ORIG == ALT{0}_CBSA) * c_destchoice_cbsa_dist * ALT{0}_DIST + \
        \
    c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + \
        \
    c_destchoice_age_18_34 * AGE_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * AGE_35_64 * ALT{0}_35_64 + c_destchoice_age_over_65 * AGE_OVER_65 * ALT{0}_65 / ALT{0}_POP + \
        \
    ALT{0}_ENT / (ALT{0}_EMP + 1) * (c_destchoice_entscore_18_34 * AGE_18_34 + c_destchoice_entscore_35_64 * AGE_35_64 + c_destchoice_entscore_65 * AGE_OVER_65) + \
        \
    IN_LF * (c_destchoice_unemp_rate * ALT{0}_UNEMP_PCT + c_destchoice_pincp * ALT{0}_PINCP) + \
        \
    (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
    (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
    (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
        \
    (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER_JOB) * ALT{0}_OWN_JOB / (ALT{0}_EMP + 1) + c_destchoice_military * MILITARY * ALT{0}_MIL_NUM + \
        \
    c_destchoice_birthstate * (POBP == ALT{0}_State) * (CBSA_NAME_ORIG != ALT{0}_CBSA) + c_destchoice_samecbsa * (CBSA_NAME_ORIG == ALT{0}_CBSA) + c_destchoice_samestate * (MIGSP == ALT{0}_State)".format(num)
    exec(initialization)
print(V100)

# full model specification (takes a bit longer to run):
# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP + c_destchoice_unemp * ALT{0}_UNEMP + c_destchoice_pincp * ALT{0}_PINCP + \
# (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
# (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
# (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
# (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER) * ALT{0}_OWN_JOB / ALT{0}_EMP".format(num)

# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_adm * ADM * ALT{0}_ADM/ALT{0}_EMP + c_agr * AGR * ALT{0}_AGR/ALT{0}_EMP + c_con * CON * ALT{0}_CON/ALT{0}_EMP + c_edu * EDU * ALT{0}_EDU/ALT{0}_EMP + c_ent * ENT * ALT{0}_ENT/ALT{0}_EMP + c_ext * EXT * ALT{0}_EXT/ALT{0}_EMP + c_fin * FIN * ALT{0}_FIN/ALT{0}_EMP + c_fod * FOD * ALT{0}_FOD/ALT{0}_EMP + c_inf * INF * ALT{0}_INF/ALT{0}_EMP + c_med * MED * ALT{0}_MED/ALT{0}_EMP + c_mfg * MFG * ALT{0}_MFG/ALT{0}_EMP + c_mng * MNG * ALT{0}_MNG/ALT{0}_EMP + c_prf * PRF * ALT{0}_PRF/ALT{0}_EMP + c_pub * PUB * ALT{0}_PUB/ALT{0}_EMP + c_rel * REL * ALT{0}_REL/ALT{0}_EMP + c_ret * RET * ALT{0}_RET/ALT{0}_EMP + c_srv * SRV * ALT{0}_SRV/ALT{0}_EMP + c_trn * TRN * ALT{0}_TRN/ALT{0}_EMP + c_utl * UTL * ALT{0}_UTL/ALT{0}_EMP + c_whl * WHL * ALT{0}_WHL/ALT{0}_EMP"

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

((((((((((((((((((log(ALT100_POP) + ((CBSA_NAME_ORIG != ALT100_CBSA) * ((c_destchoice_dist(0) * ALT100_DIST) + (c_destchoice_logdist(0) * log((ALT100_DIST + `1`)))))) + (((CBSA_NAME_ORIG == ALT100_CBSA) * c_destchoice_cbsa_dist(0)) * ALT100_DIST)) + (c_destchoice_hhcost(0) * ALT100_HH_COST)) + ((c_destchoice_college(0) * IN_COLLEGE) * ALT100_COLLEGE)) + (((c_destchoice_foreign(0) * FOREIGN) * ALT100_FOREIGN) / ALT100_POP)) + (((c_destchoice_age_18_34(0) * AGE_18_34) * ALT100_18_34) / ALT100_POP)) + ((c_destchoice_age_35_64(0) * AGE_35_64) * ALT100_35_64)) + (((c_destchoice_age_over_65(0) * AGE_OVER_65) * ALT100_65) / ALT100_POP)) + ((ALT100_ENT / (ALT100_EMP + `1`)) * (((c_destchoice_entscore_18_34(0) * AGE_18_34) + (c_destchoice_entscore_35_64(0) * AGE_35_64)) + (c_destchoice_entscore_65(0) * AGE_OVER_65)))) + (IN_LF * ((c_destchoice_unemp(0) * ALT100_UNEMP_PCT) + (c_destchoice_pincp(0) * ALT100_PINCP)))) + ((((c_destchoice_T34_T34(0) * T34) + (c_destchoice_Metro_T34(0) * METRO)) + (c

In [105]:
W0 = PWGTP * 1

In [106]:
utilities = {}
for i in range(101):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)

In [107]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-07-04]\n[16:33:46] < General >   Remove 942 unused variables from the database as only 1942 are used.\n[16:34:42] < Detailed >  Log likelihood (N = 63272):  -945637.4\n[16:34:42] < Detailed >  ** Optimization: Newton with trust region for simple bounds\n[17:03:53] < General >   Log likelihood (N = 63272):  -945637.4 Gradient norm:      1e+11 Hessian norm:       9e+16 \n[17:05:33] < Detailed >  Log likelihood (N = 63272): -2.368983e+09\n[17:05:33] < Detailed >  1 f=  14.94559 projected rel. grad.=1.4e+05 delta=   0.5 rho= -0.18 -\n[17:07:15] < Detailed >  Log likelihood (N = 63272): -1.187394e+09\n[17:07:15] < Detailed >  2 f=  14.94559 projected rel. grad.=1.4e+05 delta=  0.25 rho= -0.23 -\n[17:08:59] < Detailed >  Log likelihood (N = 63272): -5.95922e+08\n[17:08:59] < Detailed >  3 f=  14.94559 projected rel. grad.=1.4e+05 delta=  0.12 rho= -0.27 -\n[17:10:43] < Detailed >  Log likelihood (N = 63272): -2.981182e+08\n[17:10:43] < Detailed >  4 f= 

In [108]:
import biogeme.optimization as opt

In [110]:
# Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
V = utilities

# Associate the availability conditions with the alternatives
# for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
av = {}
for i in range(0, 101):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOSEN)

# formulas = {"loglike": logprob, "weight": W0}

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, suggestScales=False)
biogeme.modelName = 'us_full'

biogeme.saveIterations = False

# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[19:33:42] < General >   Remove 942 unused variables from the database as only 1942 are used.
[19:34:40] < Detailed >  Log likelihood (N = 63272):  -130078.9
[19:34:40] < Detailed >  ** Optimization: Newton with trust region for simple bounds
[20:02:20] < General >   Log likelihood (N = 63272):  -130078.9 Gradient norm:      1e+11 Hessian norm:       1e+17 
[20:04:16] < Detailed >  Log likelihood (N = 63272):  -42479.19
[20:30:56] < General >   Log likelihood (N = 63272):  -42479.19 Gradient norm:      4e+09 Hessian norm:       1e+16 
[20:30:56] < Detailed >  1 f= 0.6713742 projected rel. grad.=2.8e+04 rel. change=     1 delta=     1 rho=  0.71 +
[20:32:53] < Detailed >  Log likelihood (N = 63272):  -28638.78
[20:59:02] < General >   Log likelihood (N = 63272):  -28638.78 Gradient norm:      2e+09 Hessian norm:       8e+15 
[20:59:02] < Detailed >  2 f= 0.4526295 projected rel. grad.=1.7e+04 rel. change=   0.5 delta=     2 rho=     1 ++
[21:00:58] < Detailed >  Log likelihood (N = 6327

In [ ]:
# occupation-based coefficients

c_adm = Beta("c_adm", 0, None, None, 0)
c_agr = Beta("c_agr", 0, None, None, 0)
c_con = Beta("c_con", 0, None, None, 0)
c_edu = Beta("c_edu", 0, None, None, 0)
c_ent = Beta("c_ent", 0, None, None, 0)
c_ext = Beta("c_ext", 0, None, None, 0)
c_fin = Beta("c_fin", 0, None, None, 0)
c_fod = Beta("c_fod", 0, None, None, 0)
c_inf = Beta("c_inf", 0, None, None, 0)
c_med = Beta("c_med", 0, None, None, 0)
c_mfg = Beta("c_mfg", 0, None, None, 0)
c_mng = Beta("c_mng", 0, None, None, 0)
c_prf = Beta("c_prf", 0, None, None, 0)
c_pub = Beta("c_pub", 0, None, None, 0)
c_rel = Beta("c_rel", 0, None, None, 0)
c_ret = Beta("c_ret", 0, None, None, 0)
c_srv = Beta("c_srv", 0, None, None, 0)
c_trn = Beta("c_trn", 0, None, None, 0)
c_utl = Beta("c_utl", 0, None, None, 0)
c_whl = Beta("c_whl", 0, None, None, 0)

In [ ]:
# Staying Choice Parameters with starting values of 0
c_stay_married = Beta("c_stay_married", 0, None, None, 0)
c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
c_stay_edu_beyondhs = Beta('c_stay_edu_beyondhs', 0, None, None, 0)
c_stay_child = Beta("c_stay_child", 0, None, None, 0)
c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
c_stay = Beta("c_stay", 15, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
c_stay_college = Beta("c_stay_college", 0, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)
c_stay_mil = Beta("c_stay_mil", 0, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 0, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0, None, None, 0)
c_stay_income = Beta("c_stay_income", 0, None, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, None, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, None, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, None, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, None, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, None, None, 0)
c_stay_partcp = Beta("c_stay_partcp", 0, None, None, 0)
c_stay_rentcost = Beta("c_stay_rentcost", 0, None, None, 0)
c_stay_owncost = Beta("c_stay_owncost", 0, None, None, 0)

In [ ]:
# Bounded staying choice parameters
c_stay_income = Beta("c_stay_income", 0, 0, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, 0, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, 0, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, 0, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, 0, None, 0)

In [ ]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", 0, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0, None, None, 0)

c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 0, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 0, None, None, 0)

c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 0, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", 0, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 0, None, None, 0)

c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 0, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 0, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 0, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 0, None, None, 0)

c_destchoice_samestate = Beta("c_destchoice_samestate", 0, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", 0, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0, None, None, 0)